In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
mon = pd.read_csv('monday.csv', sep=';', parse_dates=True)
tues = pd.read_csv('tuesday.csv', sep=';', parse_dates=True)
wed = pd.read_csv('wednesday.csv', sep=';', parse_dates=True)
thurs = pd.read_csv('thursday.csv', sep=';', parse_dates=True)
fri =  pd.read_csv('friday.csv', sep=';', parse_dates=True)

FileNotFoundError: [Errno 2] No such file or directory: 'monday.csv'

In [ ]:
mon["customer_no"] = "mon_" + mon["customer_no"].astype(str)

In [ ]:
mon

In [ ]:
mon.tail(40)

In [ ]:
tues["customer_no"] = "tues_" + tues["customer_no"].astype(str)
wed["customer_no"] = "wed_" + wed["customer_no"].astype(str)
thurs["customer_no"] = "thurs_" + thurs["customer_no"].astype(str)
fri["customer_no"] = "fri_" + fri["customer_no"].astype(str)

In [ ]:
#mon = mon.set_index('timestamp')
#mon.index = pd.to_datetime(mon.index)

In [ ]:
mon

In [ ]:
tues

In [ ]:
#tues = tues.set_index('timestamp')
#tues.index = pd.to_datetime(tues.index)
#wed = wed.set_index('timestamp')
#wed.index = pd.to_datetime(wed.index)
#thurs = thurs.set_index('timestamp')
#thurs.index = pd.to_datetime(thurs.index)
#fri = fri.set_index('timestamp')
#fri.index = pd.to_datetime(fri.index)

In [ ]:
#monday_grouped = mon.groupby('customer_no').resample('1T').ffill()

In [ ]:
#monday_grouped


In [ ]:
#monday_grouped.loc[monday_grouped['customer_no'] == 'mon_1447']

In [ ]:
loc = mon.groupby('location').count()

In [ ]:
loc

In [ ]:
def count_location(df):
    return df.groupby('location').count()

In [ ]:
count_location(tues)

In [ ]:
count_location(wed)

In [ ]:
count_location(thurs)

In [ ]:
count_location(fri)

In [ ]:
mon_tue = count_location(mon) + count_location(tues)

In [ ]:
mon_tue

In [ ]:
### Calculate total over time ###

weekly_total = count_location(mon) + count_location(tues) + count_location(wed) + count_location(thurs) + count_location(fri)

In [ ]:
weekly_total

In [ ]:
mon.loc[mon['location'] == 'checkout']

In [ ]:
#def late_checkout(df):
    #finished_customers = []
    #late_customers = []
    #for customer_no in df:
        #if df.loc[df['location'] == 'checkout']:
            #finished_customers.append('customer_no')
        #else:
            #late_customers.append('customer_no')
    #return late_customers

In [ ]:
### create a function to identify late customers ###

import time, datetime
from datetime import datetime, timedelta

def late_checkout(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    timestamp =  df['timestamp'].iloc[-1] + timedelta(minutes = 1)
    
    data_checkout = df.loc[df['location'] == 'checkout']
    customers_with_checkout = data_checkout['customer_no'].unique()
    customer_ids = df['customer_no'].unique()
    customers_without_checkout = np.setxor1d(customers_with_checkout, customer_ids)
    # function which compares arrays for mismatching values
    
    for ids in customers_without_checkout:
        new_row = pd.DataFrame({'timestamp':timestamp, 'customer_no':ids, 'location':'checkout'}, index =[0])
        df = pd.concat([df, new_row]).reset_index(drop = True)
    
    return df

In [ ]:
late_checkout(mon)
late_checkout(tues)
late_checkout(wed)
late_checkout(thurs)
late_checkout(fri)

In [ ]:
### turn index to datetimeindex ###

def datetimeindex(df):
    df = df.set_index('timestamp')
    df.index = pd.to_datetime(df.index)

In [ ]:
time_of_entry = mon.groupby('customer_no')['timestamp'].first()

In [ ]:
time_of_entry.shape

In [ ]:
time_of_entry

In [ ]:
timestamp = time_of_entry - timedelta(minutes=1)

In [ ]:
timestamp

In [ ]:
### create entry point ###

def customer_entry(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    time_of_entry = df.groupby('customer_no')['timestamp'].first()
    timestamp = time_of_entry - timedelta(minutes=1)
    customer_ids = df['customer_no'].unique()
    
    for ids in customer_ids:
        new_row = pd.DataFrame({'timestamp':timestamp, 'customer_no':ids, 'location':'entry'}, index =[0])
        df = pd.concat([df, new_row]).reset_index(drop = True)
    
    return df

In [ ]:
def customer_enters(df):
    df_entry = pd.DataFrame(df.groupby('customer_no').first()['timestamp'] - timedelta(minutes=1))
    df_entry['location'] = 'entry'
    df_entry.reset_index(inplace=True)
    
    df_with_entry = pd.concat([df, df_entry]).sort_values(['timestamp','location'])
    
    return df_with_entry

In [ ]:
test = customer_enters(mon)
test = late_checkout(test)

In [ ]:
mon.loc[mon['customer_no'] == 'mon_1447']

In [ ]:
### turn index to datetimeindex ###

def datetimeindex(df):
    df = df.set_index('timestamp')
    df.index = pd.to_datetime(df.index)
    return df

In [ ]:
mon.loc[mon['location'] != 'checkout'].groupby('customer_no').tail()

In [ ]:
test = datetimeindex(test)

In [ ]:
test.tail(20)

In [ ]:
tues = customer_enters(tues)
tues = late_checkout(tues)
tues = datetimeindex(tues)

In [ ]:
tues

In [ ]:
wed = customer_enters(wed)
wed = late_checkout(wed)
wed = datetimeindex(wed)

In [ ]:
thurs = customer_enters(thurs)
thurs = late_checkout(thurs)
thurs = datetimeindex(thurs)

In [ ]:
fri = customer_enters(fri)
fri = late_checkout(fri)
fri = datetimeindex(fri)

In [ ]:
fri

In [ ]:
week = pd.concat([test, tues, wed, thurs, fri]).sort_values(['timestamp','location'])

In [ ]:
week

In [1]:
weekb = week.groupby('id').resample('1T').first()

NameError: name 'week' is not defined